In [4]:
from lxml import etree

In [5]:

path_in = "/home/ziane212/Téléchargements/1487_Cronicque_convTags_01-03-23_ambi_RevNR.xml"                  

path_out = "/home/ziane212/Téléchargements/1487_Cronicque_convTags_01-03-23_ambi_RevNR_test.xml"

In [7]:
tree = etree.parse(open(path_in, encoding='utf-8'))
tree_tokenized = tree.getroot()

test_ = []
l_numSG = []

for id_, s in enumerate(tree_tokenized.findall('.//s')):
    
    for w in s.findall('.//w'):
        
        if w.get('udpos') ==  'NOUN' and w.get('function') ==  'nsubj':
            id_gov = w.get('n')
            
            for w2 in tree_tokenized.findall('.//s')[id_]:
                
                id_dep = w2.get('n')
                
                var = False
                
                if w2.get('head') == id_gov and w2.get('udpos') == 'det':
                    
                    var = True
                                
                if var == False:
                    
                        
                
               

In [8]:
def indent(elem, level=0, more_sibs=False):
    i = "\n"
    if level:
        i += (level-1) * '\t'
    num_kids = len(elem)
    if num_kids:
        if not elem.text or not elem.text.strip():
            elem.text = i + "\t"
            if level:
                elem.text += '\t'
        count = 0
        for kid in elem:
            indent(kid, level+1, count < num_kids - 1)
            count += 1
        if not elem.tail or not elem.tail.strip():
            elem.tail = i
            if more_sibs:
                elem.tail += '\t'
    else:
        if level and (not elem.tail or not elem.tail.strip()):
            elem.tail = i
            if more_sibs:
                elem.tail += '\t'

In [9]:
indent(tree_tokenized)
#etree.dump(new_root)
etree.ElementTree(tree_tokenized).write(path_out, encoding='utf-8')